In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
card_holders = pd.read_csv('/content/drive/MyDrive/Preppin Data/Data/2024/Flow Card Holders.csv')
non_holders = pd.read_csv('/content/drive/MyDrive/Preppin Data/Data/2024/Non Flow Card Holders.csv')
target_file = pd.ExcelFile('/content/drive/MyDrive/Preppin Data/Data/2024/PD 2024 Wk 3 Input.xlsx')

In [ ]:
dfs = {}
for sheet in target_file.sheet_names:
  dfs[sheet] = pd.read_excel(target_file, sheet_name=sheet)

quarter1 = dfs['Q1']
quarter2 = dfs['Q2']
quarter3 = dfs['Q3']
quarter4 = dfs['Q4']

In [ ]:
rename_class = {'First Class':'Economy', 'Business Class':'Premium Economy', 'Premium Economy':'Business Class', 'Economy':'First Class'}
abbreviate_class = {'Economy':'E', 'Premium Economy':'PE', 'Business Class':'BC', 'First Class':'FC'}

In [ ]:
card_holders.Class = card_holders.Class.map(rename_class)
card_holders.Class = card_holders.Class.map(abbreviate_class)
non_holders.Class = non_holders.Class.map(rename_class)
non_holders.Class = non_holders.Class.map(abbreviate_class)

In [ ]:
card_holders.Date = pd.to_datetime(card_holders.Date)
non_holders.Date = pd.to_datetime(non_holders.Date)

In [ ]:
card_holders['Month'] = card_holders.Date.dt.month
non_holders['Month'] = non_holders.Date.dt.month

In [ ]:
card_holders_sales = card_holders.groupby(['Class', 'Month'])['Price'].sum().reset_index().rename(columns={'Price':'Sales'})
non_holders_sales = non_holders.groupby(['Class', 'Month'])['Price'].sum().reset_index().rename(columns={'Price':'Sales'})

In [ ]:
target = pd.concat([quarter1, quarter2, quarter3, quarter4], axis=0)
sales = pd.concat([card_holders_sales, non_holders_sales], axis=0)
sales = sales.groupby(['Class', 'Month'])['Sales'].sum().reset_index()

In [ ]:
output_df = pd.merge(sales, target, how='left', on=['Class', 'Month'])

In [ ]:
output_df['Difference to Target'] = output_df.Sales - output_df.Target

In [ ]:
output_df.head()
output_df.shape

,Class,Month,Sales,Target,Difference to Target
0,BC,1,67297.5,85000,-17702.5
1,BC,2,69222.5,86000,-16777.5
2,BC,3,84357.5,87000,-2642.5
3,BC,4,81522.5,88000,-6477.5
4,BC,5,85065.0,89000,-3935.0


(48, 5)